<a href="https://colab.research.google.com/github/farhankhot/Cats_Predictor/blob/main/Cats_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install flask-ngrok

In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# Write file to hosted runtime in Colab  https://stackoverflow.com/questions/21034373/how-to-load-edit-run-save-text-files-py-into-an-ipython-notebook-cell

%%writefile model.py

import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# MAKE PRODUCTION READY
# Connect Colab with gdrive: https://www.marktechpost.com/2019/06/07/how-to-connect-google-colab-with-google-drive/
data_dir = '/content/gdrive/MyDrive/Cats/photos'

# Connect photos to code by path https://stackoverflow.com/questions/57443122/how-to-load-images-in-google-colab-notebook-using-tensorflow-from-mounted-google
data_dir = pathlib.Path(data_dir)

# TRY CODE
# roses = list(data_dir.glob('roses/*'))
# PIL.Image.open(str(roses[0]))

# ----------DEFINE PARAMETERS FOR THE LOADER--------
batch_size = 32
img_height = 180
img_width = 180

# ---------SPLITTING TRAINING AND TEST DATA---------

# DEF: image_dataset_from_directory: Generates a tf.data.Dataset from image files in a directory.

# tf.keras.preprocessing.image_dataset_from_directory(
#     directory, labels='inferred', label_mode='int',
#     class_names=None, color_mode='rgb', batch_size=32, image_size=(256,
#     256), shuffle=True, seed=None, validation_split=None, subset=None,
#     interpolation='bilinear', follow_links=False, smart_resize=False
# )
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = 'training',
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

def get_class_names():
  class_names = train_ds.class_names
  return class_names




if __name__ == "__main__":
  # ---------------DISPLAYING FIRST 9 IMAGES-------------
  # plt.figure(figsize=(10,10))
  # for images, labels in train_ds.take(1):
  #   for i in range(9):
  #     ax = plt.subplot(3, 3, i+1)
  #     plt.imshow(images[i].numpy().astype("uint8"))
  #     plt.title(class_names[labels[i]])
  #     plt.axis("off")

  AUTOTUNE = tf.data.AUTOTUNE
  train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
  val_ds = val_ds.cache().prefetch(buffer_size = AUTOTUNE)

  # ----------STANDARDIZE VALUES FROM [0,255] to [0, 1]------------
  normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

  # -------------OVERFITTING FIX------------
  # DATA AUGMENTATION: Data augmentation takes the approach of generating additional training data from your existing examples by augmenting them using random transformations that yield believable-looking images. 
  #                    This helps expose the model to more aspects of the data and generalize better.
  # DROPOUt is also employed
  data_augmentation = keras.Sequential([
                                        layers.experimental.preprocessing.RandomFlip("horizontal", input_shape = (img_height, img_width, 3) ),
                                        layers.experimental.preprocessing.RandomRotation(0.1),
                                        layers.experimental.preprocessing.RandomZoom(0.1),
  ])


  # -----------CREATE MODEL---------------
  num_classes = 12 
  model = Sequential ([
    data_augmentation,
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'), # Layer 1
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'), # Layer 2
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'), # Layer 3
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
  ])

  # -------------COMPILE MODEL------------
  model.compile(optimizer = 'adam', loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


  # -------------TRAIN MODEL--------------
  epochs = 10
  history = model.fit(train_ds, validation_data = val_ds, epochs=epochs)

  # Save model to drive: https://www.tensorflow.org/tutorials/keras/save_and_load#save_the_entire_model
  model.save('/content/gdrive/MyDrive/Cats/model/my_model.h5')

  # View the model
  # new_model.summary()

  # --------------CHECKING MODEL STATS------------
  # acc = history.history['accuracy']
  # val_acc = history.history['val_accuracy']

  # loss = history.history['loss']
  # val_loss = history.history['val_loss']

  # epochs_range = range(epochs)

  # plt.figure(figsize=(8, 8))
  # plt.subplot(1, 2, 1)
  # plt.plot(epochs_range, acc, label='Training Accuracy')
  # plt.plot(epochs_range, val_acc, label='Validation Accuracy')
  # plt.legend(loc='lower right')
  # plt.title('Training and Validation Accuracy')

  # plt.subplot(1, 2, 2)
  # plt.plot(epochs_range, loss, label='Training Loss')
  # plt.plot(epochs_range, val_loss, label='Validation Loss')
  # plt.legend(loc='upper right')
  # plt.title('Training and Validation Loss')
  # plt.show()



Writing model.py


In [11]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify, render_template
from werkzeug.utils import secure_filename

import os
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# FLask on Google Colab: https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b

# BAD IMPORT, CHANGE TO PRODUCTION LOCATION
# Import only the function (still causes model.py to run): https://stackoverflow.com/questions/6523791/why-is-python-running-my-module-when-i-import-it-and-how-do-i-stop-it
from model import get_class_names

# To add flask_ngrok: https://colab.research.google.com/notebooks/snippets/importing_libraries.ipynb

# Running flask in repl.it: https://stackoverflow.com/questions/30554702/cant-connect-to-flask-web-service-connection-refused
app = Flask(__name__, static_folder='/content/gdrive/MyDrive/Cats/static', template_folder='/content/gdrive/MyDrive/Cats/template')

# Use app.config to specify upload folder: https://flask.palletsprojects.com/en/2.0.x/patterns/fileuploads/ 
app.config['UPLOAD_FOLDER'] = '/content/gdrive/MyDrive/Cats/user_inputs'
run_with_ngrok(app)

def predict(image):

  # https://www.tensorflow.org/tutorials/keras/save_and_load#save_the_entire_model
  new_model = tf.keras.models.load_model('/content/gdrive/MyDrive/Cats/model/my_model.h5', custom_objects={'tf': tf})

  # ----------------CHECK WITH SAMPLE IMAGE------------------

  # cat_path = '/content/gdrive/MyDrive/photos/samples/sample_cat_3.jpg'
  # cat_path = pathlib.Path(cat_path)

  cat_path = image

  img = keras.preprocessing.image.load_img(cat_path, target_size=(180, 180) ) # Change to img_height, img_width someday

  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array,0)

  predictions = new_model.predict(img_array)
  score = tf.nn.softmax(predictions[0]) # get first prediction
  class_names = get_class_names()
  np.argmax(score)

  # Formatting: https://stackoverflow.com/questions/17153779/how-can-i-print-variable-and-string-on-same-line-in-python
  print("This image most likely belongs to {} with a {:.2f} percent confidence.".format(class_names[np.argmax(score)], 100 * np.max(score)))
  
  ans = class_names[np.argmax(score)]
  
  return ans, 100 * np.max(score)

@app.route('/', methods=['GET', 'POST'])
def home():

  # POST and GET in same route: https://stackoverflow.com/questions/42018603/handling-get-and-post-in-same-flask-view

  if request.method == 'POST':
    data = request.files['postPicture']
    # Save to Google Drive: https://stackoverflow.com/questions/42424853/saving-upload-in-flask-only-saves-to-project-root
    data.save(os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(data.filename))) # WTF 
    # print(data.read)

    # Formatting: https://stackoverflow.com/questions/17153779/how-can-i-print-variable-and-string-on-same-line-in-python
    x = '/content/gdrive/MyDrive/Cats/user_inputs/{}'.format(data.filename) 
    
    name, confidence = predict(x)  
    
    return "This image most likely belongs to {} with a {:.2f} percent confidence.".format(name, confidence)
  else:
	  return app.send_static_file('index.html')

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://11ddeaae8915.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Aug/2021 11:58:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2021 11:58:51] "GET /static/script.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2021 11:58:51] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Aug/2021 11:58:55] "POST / HTTP/1.1" 200 -


This image most likely belongs to Bengal with a 87.13 percent confidence.


127.0.0.1 - - [01/Aug/2021 11:58:58] "GET / HTTP/1.1" 200 -
